# .py로 바꿔서 제출

In [1]:
from PIL import Image
import numpy as np
import math

## 1-1)

In [2]:
# %%
# 1
def boxfilter(n):
    # n % 2 == 1이면 통과, 아니면 Error 문구 출력하고 에러 처리
    assert n % 2 == 1, "Dimension must be odd"
    # numpy 이용해서 행렬 형태 만들어준다.
    ret = (1/(n**2))*np.ones([n, n])
    return ret


In [3]:
boxfilter(3)

array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]])

In [4]:
boxfilter(4)

AssertionError: Dimension must be odd

In [ ]:
boxfilter(7)

# 1-2)

In [5]:
# %%
# 2
def gauss1d(sigma):
    # sigma value가 0이 되서는 안된다.(sigma value가 0이면 분모가 0이 된다.)
    assert sigma != 0, "Sigma value sholudn't be zero"
    
    # sigma를 6으로 곱하고 올림
    length = int(math.ceil(float(sigma) * 6))
    
    # 짝수라면 홀수로 바꾸기 위해서 +1
    if length % 2 == 0:
        length += 1
    
    center = length // 2
    
    # -center ~ center의 1차원 배열 생성
    x = np.arange(-center, center + 1)
    
    # sum to 1을 위한 처리
    ret = np.exp(-(x**2)/(2*(sigma**2)))
    ret /= np.sum(ret)
    
    return ret

In [6]:
gauss1d(0.3)

array([0.00383626, 0.99232748, 0.00383626])

In [7]:
gauss1d(0.5)

array([0.10650698, 0.78698604, 0.10650698])

In [8]:
gauss1d(1)

array([0.00443305, 0.05400558, 0.24203623, 0.39905028, 0.24203623,
       0.05400558, 0.00443305])

In [9]:
gauss1d(2)

array([0.0022182 , 0.00877313, 0.02702316, 0.06482519, 0.12110939,
       0.17621312, 0.19967563, 0.17621312, 0.12110939, 0.06482519,
       0.02702316, 0.00877313, 0.0022182 ])

# 1-3)

In [10]:
# %%    
# 3
def gauss2d(sigma):
    # np.outer(외적) 연산
    ret = np.outer(gauss1d(sigma), gauss1d(sigma))
    ret /= np.sum(ret)
    return ret


In [11]:
gauss2d(0.5)

array([[0.01134374, 0.08381951, 0.01134374],
       [0.08381951, 0.61934703, 0.08381951],
       [0.01134374, 0.08381951, 0.01134374]])

In [12]:
gauss2d(1)

array([[1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [1.76900911e-03, 2.15509428e-02, 9.65846250e-02, 1.59241126e-01,
        9.65846250e-02, 2.15509428e-02, 1.76900911e-03],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05]])

In [13]:
# %%
# 4
# np.float32의 입력 변수
def convolve2d(array, filter):
    # np.flipud : 위, 아래 방향 뒤집기
    # np.fliplr : 좌, 우 방향 뒤집기
    
    filter = np.flipud(np.fliplr(filter))
    
    pad = int(np.sqrt(filter.size)) # 19
    pad2 = pad//2 # 9
    
    # 리턴값
    output = np.zeros_like(array, dtype = 'float32')
    
    # Zero padding
    padded = np.zeros((array.size[1] + 2*pad2, array.size[0] + 2*pad2))
    padded[pad2:-pad2, pad2:-pad2] = array
    
    # 이중 for loop로 zero padding 구현
    for x in range(array.size[0]):
        for y in range(array.size[1]):
            output[y,x] = (filter * padded[y:y+pad, x:x+pad]).sum()
            
    return output
    
from scipy import signal # 테스트 용도
def gaussconvolve2d(array, sigma):
    # sigma를 통해 gauss를 통해 convolution 연산을 통해 결과를.
    # convolve2d와 gauss2d를 이용해서 만들면 된다.
    filter = gauss2d(sigma)
    # output = signal.convolve2d(array, filter, 'same') # 테스트 용도
    output = convolve2d(array, filter)
    
    return output

# gauss2d를 이용하기 위해서 grayscale로 변환해준다음에 적용해야 한다.
# 배열을 float32로 변환해준 후, 다시 정수형으로 바꿔서 이미지파일로 변환해야함.

# PIL을 이용해서 결과(이미지)를 보여주면 된다
    

## Original Image

In [14]:
# dog사진을 gaussconvolve2d를 통해 나온 결과값을 출력하면 된다.
im = Image.open('Image/hw2_image/2b_dog.bmp')
print(type(im))
# color to grayscale
im = im.convert('L')
im.show()


<class 'PIL.BmpImagePlugin.BmpImageFile'>


## filtered Image

In [15]:
output = gaussconvolve2d(im, 3)
print(output)
# PIL로 출력하기 위해서 정수형 'uint8'로 변경
print(type(output))
output = output.astype('uint8')
output2 = Image.fromarray(output)
output2.show()

[[ 81.8036   100.00801  115.4233   ...  12.830245  11.387731   9.534003]
 [ 99.94278  122.19637  141.04482  ...  15.678084  13.916371  11.651171]
 [115.23515  140.91483  162.67232  ...  18.083757  16.053658  13.441321]
 ...
 [ 32.498684  40.811886  48.47309  ...  31.991108  27.793146  22.785738]
 [ 27.935917  35.198284  41.94914  ...  26.927996  23.383022  19.160536]
 [ 22.716507  28.700226  34.300663 ...  21.51462   18.660168  15.271497]]
<class 'numpy.ndarray'>


# 2-1)
low-pass filter
sigma가 높으면 blur가 많이 되고,
sigma가 낮으면 blur가 적게 됨.
적절한 값을 조정해서 blur된 이미지가 나오도록.
여기서는 rgb채널로 분리한 다음에 gauss filter로 convolution해준 후에 rgb 채널을 합쳐서 
rgb 채널의 sigma가 동일하도록

## bicycle and motorcycle

In [193]:
# low-pass filter를 이용한 결과 + high-pass filter를 이용한 결과

In [194]:
# 각각의 pair로 이를 적용한다. 1a1b, 2a2b, 3a3b를 리포트에 작성

In [195]:
im1a = Image.open('Image/hw2_image/1a_bicycle.bmp').convert('RGB')

In [196]:
# split()을 이용해 각 채널을 나누었다.
red, green, blue = im1a.split()

In [197]:
red = gaussconvolve2d(red, 0.5)
print(red)
# PIL로 출력하기 위해서 정수형 'uint8'로 변경
red = red.astype('uint8')
red = Image.fromarray(red)
# red.show()

[[199.27127 225.18292 225.47977 ... 225.56358 225.80228 199.35509]
 [224.57492 254.27715 254.68048 ... 254.69182 254.36096 224.58627]
 [225.39595 254.66913 255.      ... 255.      254.66913 225.39595]
 ...
 [226.26675 254.78699 255.      ... 255.      254.68048 225.47977]
 [226.07642 254.57397 254.78699 ... 254.78699 254.46747 225.28943]
 [200.76143 226.07642 226.26675 ... 226.26675 225.98126 200.05826]]


In [198]:
green = gaussconvolve2d(green, 0.5)
print(red)
# PIL로 출력하기 위해서 정수형 'uint8'로 변경
green = green.astype('uint8')
green = Image.fromarray(green)
# green.show()

<PIL.Image.Image image mode=L size=453x300 at 0x1DC2B657FA0>


In [199]:
blue = gaussconvolve2d(blue, 0.5)
print(red)
# PIL로 출력하기 위해서 정수형 'uint8'로 변경
blue = blue.astype('uint8')
blue = Image.fromarray(blue)
# blue.show()

<PIL.Image.Image image mode=L size=453x300 at 0x1DC2B657FA0>


In [200]:
# merge()를 이용해 각 채널을 합쳤다.
im1a_lowfreq = Image.merge("RGB", (red, green, blue))
# im1a_lowfreq.show()

# 2-2)
high-pass

In [201]:
im1b = Image.open('Image/hw2_image/1b_motorcycle.bmp').convert('RGB')

In [202]:
red_1b, green_1b, blue_1b = im1b.split()

In [219]:
red_1b_lowfreq = gaussconvolve2d(red_1b, 2)
red_1b_lowfreq = red_1b_lowfreq.astype('uint8')
red_1b_lowfreq = Image.fromarray(red_1b_lowfreq)
# red_1b_lowfreq.show()

green_1b_lowfreq = gaussconvolve2d(green_1b, 2)
green_1b_lowfreq = green_1b_lowfreq.astype('uint8')
green_1b_lowfreq = Image.fromarray(green_1b_lowfreq)

blue_1b_lowfreq = gaussconvolve2d(blue_1b, 2)
blue_1b_lowfreq = blue_1b_lowfreq.astype('uint8')
blue_1b_lowfreq = Image.fromarray(blue_1b_lowfreq)

In [220]:
# R, G, B 각각을 다시 합친다.
im1b_lowfreq = Image.merge("RGB", (red_1b_lowfreq, green_1b_lowfreq, blue_1b_lowfreq))
im1b_lowfreq.show()

In [221]:
im1b = np.array(im1b)
im1b_lowfreq = np.array(im1b_lowfreq)

In [222]:
# 원본 이미지와 lowfreq filter를 이용하여 highfreq filter를 만들어주었다.
im1b_highfreq = im1b - im1b_lowfreq + 128

In [223]:
im1b_highfreq = Image.fromarray(im1b_highfreq)
im1b_highfreq.show()

# 2-3)
high-pass + low-pass

In [224]:
im1a_lowfreq.show()

In [225]:
im1b_highfreq.show()

In [226]:
output = np.array(im1a_lowfreq) + np.array(im1b_highfreq)
output = Image.fromarray(output)
output.show()